# Initialize


In [1]:
%load_ext autoreload
%autoreload 2


In [5]:
import torch
from torchvision import datasets, transforms
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np

from model import VAE_priorCategorical, VAE_priorHFM
import metadata as md
from datasets import Dataset_HFM, Dataset_pureHFM, load_dsprites
from utilities import sample_images, get_empirical_latent_distribution, calculate_kl_divergence_with_HFM
from peak_analysis import get_peaks_data, accumulate_peaks_data, plot_peaks_data, find_peaks_from_empirical_distribution
from find_gauge import calculate_HFM_over_disordered_states


In [3]:
device = torch.device('cpu')


# Datasets


## dSprites


In [ ]:
train_loader_dSprites, val_loader_dSprites = load_dsprites()


## MNIST


In [4]:
train_loader_MNIST = torch.utils.data.DataLoader(
    datasets.MNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=True,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=md.batch_size,
    shuffle=True
    )

val_loader_MNIST = torch.utils.data.DataLoader(
    datasets.MNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=False,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=md.batch_size,
    shuffle=False
    )


## FashionMNIST


In [ ]:
train_loader_FashionMNIST = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=True,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=md.batch_size,
    shuffle=True
    )

val_loader_FashionMNIST = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        '/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data',
        train=False,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=md.batch_size,
    shuffle=False
    )
## pureHFM
dataset_HFM_train = Dataset_pureHFM(csv_file='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/pureHFM/512features/glog2_train60000.pt',
                        root_dir='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/pureHFM')

train_loader_pureHFM = DataLoader(
    dataset_HFM_train,
    batch_size=md.batch_size,
    shuffle=True
)

dataset_HFM_val = Dataset_pureHFM(csv_file='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/pureHFM/512features/glog2_validation10000.pt',
                            root_dir='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/pureHFM')

val_loader_pureHFM = DataLoader(
    dataset_HFM_val, # Importante: usa dataset_HFM_val qui, non dataset_HFM
    batch_size=md.batch_size,
    shuffle=False
)
## expandedHFM 32-1024
dataset_HFM_train = Dataset_HFM(csv_file='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/expandedHFM/32_1024features/2hl_glog2_train60000.pt',
                        root_dir='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/pureHFM')

train_loader_expandedHFM_32_1024 = DataLoader(
    dataset_HFM_train,
    batch_size=md.batch_size,
    shuffle=True
)

dataset_HFM_val = Dataset_HFM(csv_file='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/expandedHFM/32_1024features/2hl_glog2_validation10000.pt',
                            root_dir='/Users/enricofrausin/Programmazione/PythonProjects/Fisica/data/pureHFM')

val_loader_expandedHFM_32_1024 = DataLoader(
    dataset_HFM_val, # Importante: usa dataset_HFM_val qui, non dataset_HFM
    batch_size=md.batch_size,
    shuffle=False
)
# PriorHFM, klG = log2

# PriorHFM

In [12]:
from find_gauge import flip_gauge_bits, get_feature_frequencies_sorted, reorder_bit_states_by_frequency, return_goodgauged_empirical_probs, find_optimal_g_parameter

In [15]:
debug_dict = {(1,1,0,1,0,0): 0.1,
              (0,1,0,1,0,1): 0.3,
              (1,0,0,1,1,1): 0.2,
              (1,0,1,1,0,1): 0.2,
              (1,0,1,1,1,1): 0.2}



In [16]:
return_goodgauged_empirical_probs(debug_dict)

{(1, 0, 0, 0, 1, 0): 0.1,
 (0, 0, 0, 0, 0, 0): 0.3,
 (1, 1, 0, 1, 0, 0): 0.2,
 (1, 1, 1, 0, 0, 0): 0.2,
 (1, 1, 1, 1, 0, 0): 0.2}

## train over MNIST


In [7]:
train_loader = train_loader_MNIST
val_loader = val_loader_MNIST
input_dim = 784


### latent_dim = 8


In [11]:

# hidden layer = 1
my_model = VAE_priorCategorical(input_dim=input_dim, categorical_dim=2, latent_dim=8, decrease_rate=0.5, device=device, num_hidden_layers=1, LayerNorm=True).to(device)
my_model.load_state_dict(torch.load('/Users/enricofrausin/Programmazione/PythonProjects/Fisica/Architetture/VAE/discrete/models_parameters/priorCategorical/MNIST/ld8_glog2_ep15_lmb01_dr05_gKLlog2_LN_1hl_0.pth', map_location=device))
print(calculate_HFM_over_disordered_states(my_model, train_loader, device=device))


TypeError: iteration over a 0-d tensor

In [ ]:

# hidden layer = 2
my_model = VAE_priorHFM(input_dim=input_dim, latent_dim=8, g=np.log(2), decrease_rate=0.5, device=device, num_hidden_layers=2, LayerNorm=True).to(device)
my_model.load_state_dict(torch.load('/Users/enricofrausin/Programmazione/PythonProjects/Fisica/Architetture/VAE/discrete/models_parameters/priorHFM/MNIST/ld8_glog2_ep15_lmb01_dr05_gKLlog2_NL_2hl_0.pth', map_location=device))
current_data_dict = get_peaks_data(my_model, train_loader, device, kl_args={'g': different_klG})
accumulate_peaks_data(data_dict_ld8_klg, current_data_dict)

# hidden layer = 3
my_model = VAE_priorHFM(input_dim=input_dim, latent_dim=8, g=np.log(2), decrease_rate=0.5, device=device, num_hidden_layers=3, LayerNorm=True).to(device)
my_model.load_state_dict(torch.load('/Users/enricofrausin/Programmazione/PythonProjects/Fisica/Architetture/VAE/discrete/models_parameters/priorHFM/MNIST/ld8_glog2_ep15_lmb01_dr05_gKLlog2_NL_3hl_0.pth', map_location=device))
current_data_dict = get_peaks_data(my_model, train_loader, device, kl_args={'g': different_klG})
accumulate_peaks_data(data_dict_ld8_klg, current_data_dict)

# hidden layer = 4
my_model = VAE_priorHFM(input_dim=input_dim, latent_dim=8, g=np.log(2), decrease_rate=0.5, device=device, num_hidden_layers=4, LayerNorm=True).to(device)
my_model.load_state_dict(torch.load('/Users/enricofrausin/Programmazione/PythonProjects/Fisica/Architetture/VAE/discrete/models_parameters/priorHFM/MNIST/ld8_glog2_ep15_lmb01_dr05_gKLlog2_LN_4hl_0.pth', map_location=device))
current_data_dict = get_peaks_data(my_model, train_loader, device, kl_args={'g': different_klG})
accumulate_peaks_data(data_dict_ld8_klg, current_data_dict)

# hidden layer = 5
my_model = VAE_priorHFM(input_dim=input_dim, latent_dim=8, g=np.log(2), decrease_rate=0.55, device=device, num_hidden_layers=5, LayerNorm=True).to(device)
my_model.load_state_dict(torch.load('/Users/enricofrausin/Programmazione/PythonProjects/Fisica/Architetture/VAE/discrete/models_parameters/priorHFM/MNIST/ld8_glog2_ep15_lmb01_dr055_gKLlog2_NL_5hl_0.pth', map_location=device))
current_data_dict = get_peaks_data(my_model, train_loader, device, kl_args={'g': different_klG})
accumulate_peaks_data(data_dict_ld8_klg, current_data_dict)

# hidden layer = 6
my_model = VAE_priorHFM(input_dim=input_dim, latent_dim=8, g=np.log(2), decrease_rate=0.6, device=device, num_hidden_layers=6, LayerNorm=True).to(device)
my_model.load_state_dict(torch.load('/Users/enricofrausin/Programmazione/PythonProjects/Fisica/Architetture/VAE/discrete/models_parameters/priorHFM/MNIST/ld8_glog2_ep15_lmb01_dr06_gKLlog2_NL_6hl_0.pth', map_location=device))
current_data_dict = get_peaks_data(my_model, train_loader, device, kl_args={'g': different_klG})
accumulate_peaks_data(data_dict_ld8_klg, current_data_dict)

print(data_dict_ld8_klg['peaks_number'])